In [4]:
# !pip install tensorflow librosa matplotlib joblib glob

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import librosa
import joblib
from glob import glob
import IPython.display as ipd

In [19]:
example_audio = glob('Voice_Emotion_Dataset/*/*.wav')[0]

In [20]:
# Resample the audio to a different sample rate
resampled_audio, _ = librosa.load(example_audio, sr=4000)
ipd.Audio(resampled_audio, rate=4000)
# ipd.Audio(example_audio)

Ler os arquivos de áudio para coletar os coeficientes e os os labels. Os labels serão divididos da seguinte maneira:
- Anger: 0
- Disgust: 1
- Fear: 2
- Happy: 3
- Neutral: 4
- Sad: 5

In [21]:
sr = 4000
x, sr = librosa.load(example_audio, sr=sr)
x = librosa.util.fix_length(x, size=sr*10)
print(x, sr, len(x))

librosa.feature.mfcc(y=x, sr=sr, n_mfcc=64).shape

[-2.4583949e-06 -8.6129617e-08  8.7763631e-08 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] 4000 40000


(64, 79)

In [4]:
base_dir = './Voice_Emotion_Dataset/'

In [23]:
len(glob('Voice_Emotion_Dataset/*/*.wav'))

54490

In [24]:
max_len = max([librosa.get_duration(path=audio_file) for audio_file in glob('Voice_Emotion_Dataset/*/*.wav')])
min_len = min([librosa.get_duration(path=audio_file) for audio_file in glob('Voice_Emotion_Dataset/*/*.wav')])
print('min_index: ', [librosa.get_duration(path=audio_file) for audio_file in glob('Voice_Emotion_Dataset/*/*.wav')].index(min_len))
min_len, max_len

min_index:  [3.8371666666666666, 3.83718820861678, 3.83718820861678, 4.796462585034013, 3.83718820861678, 3.8371666666666666, 3.83718820861678, 3.83718820861678, 4.796462585034013, 3.83718820861678, 3.6036041666666665, 3.603628117913832, 3.603628117913832, 4.50453514739229, 3.603628117913832, 3.370041666666667, 3.3700680272108845, 3.3700680272108845, 4.212607709750567, 3.3700680272108845, 3.2699375, 3.2699773242630386, 3.2699773242630386, 4.087482993197279, 3.2699773242630386, 3.5702291666666666, 3.570249433106576, 3.570249433106576, 4.46281179138322, 3.570249433106576, 3.536875, 3.5369160997732427, 3.5369160997732427, 4.421133786848072, 3.5369160997732427, 4.204208333333334, 4.20421768707483, 4.20421768707483, 5.255283446712018, 4.20421768707483, 3.2699375, 3.2699773242630386, 3.2699773242630386, 4.087482993197279, 3.2699773242630386, 3.3032916666666665, 3.3033106575963718, 3.3033106575963718, 4.129160997732426, 3.3033106575963718, 3.3366666666666664, 3.336689342403628, 3.336689342403

(1.254075530433358, 6.38140589569161)

In [29]:
label_map = {'anger': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5}
lst = []
number_of_audios_per_class = -1 # set -1 for all audios
sr = 4000
n_mfcc = 64

for emotion in label_map.keys():
    folder_path = os.path.join(base_dir, emotion)
    audios_number = 0
    # Iterate over the audio files in the folder
    for audio_file in glob(base_dir + emotion + '/*.wav'):
        # print(os.path.join(folder_path, audio_file))
        x, sample_rate = librosa.load(audio_file, sr=sr)
        x = librosa.util.fix_length(x, size=int(sr*max_len))
        mfccs = np.array(librosa.feature.mfcc(y=x, sr=sr, n_mfcc=n_mfcc))
        label = label_map[emotion]
        arr = mfccs, label
        lst.append(arr)
        audios_number += 1
        if(audios_number == number_of_audios_per_class): break
        print(f'\r{audios_number} {emotion} audio files loaded', end='')
    print(audios_number, emotion, 'audio files loaded')

9315 anger audio files loaded9315 anger audio files loaded
9315 disgust audio files loaded9315 disgust audio files loaded
9315 fear audio files loaded9315 fear audio files loaded
9315 happy audio files loaded9315 happy audio files loaded
7915 neutral audio files loaded7915 neutral audio files loaded
9315 sad audio files loaded9315 sad audio files loaded


In [30]:
x, y = zip(*lst)

In [34]:
x = np.asarray(x)
y = np.asarray(y)

((64, 50), (54490, 64, 50), (54490,))

In [2]:
import joblib

In [36]:


X_name = 'x.joblib'
y_name = 'y.joblib'
save_dir = base_dir

savedX = joblib.dump(x, os.path.join(save_dir, X_name))
savedy = joblib.dump(y, os.path.join(save_dir, y_name))

In [5]:
x = joblib.load(base_dir + 'x.joblib')
y = joblib.load(base_dir + 'y.joblib')

In [12]:
input_shape = x[0].shape
input_shape, x.shape, y.shape

((64, 50), (54490, 64, 50), (54490,))

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

## Neural Network

In [7]:
# x_traincnn = np.expand_dims(X_train, axis=2)
# x_testcnn = np.expand_dims(X_test, axis=2)

x_traincnn = X_train
x_testcnn = X_test

In [8]:
x_traincnn.shape, x_testcnn.shape

((36508, 64, 50), (17982, 64, 50))

In [9]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# List available GPUs
print("Available GPUs: ", tf.config.experimental.list_physical_devices('GPU'))

Num GPUs Available:  0
Available GPUs:  []


In [46]:
callback = keras.callbacks.EarlyStopping(monitor='loss',patience= 3)
# callbacks = [
#     keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=10),
# ]
model = Sequential()
model.add(Conv1D(64, 5, padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(6))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy'])

cnn_history = model.fit(x_traincnn, y_train,
                batch_size=16, epochs=1000,
                validation_data=(x_testcnn, y_test), callbacks=[callback])

c:\Users\alech\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.3919 - loss: 8.7454 - val_accuracy: 0.4783 - val_loss: 1.8423
Epoch 2/1000
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.4913 - loss: 1.9411 - val_accuracy: 0.4642 - val_loss: 2.0993
Epoch 3/1000
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.5159 - loss: 1.7005 - val_accuracy: 0.5034 - val_loss: 1.6687
Epoch 4/1000
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.5295 - loss: 1.5918 - val_accuracy: 0.5161 - val_loss: 1.8609
Epoch 5/1000
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.5426 - loss: 1.5473 - val_accuracy: 0.5269 - val_loss: 1.4954
Epoch 6/1000
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.5468 - loss: 1.5085 - val_accuracy: 0.5415 - val_loss: 1.5163
Epoch 7/1000
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.5560 - loss: 1.4785 - val_accuracy: 0.5266 - val_loss: 1.8239
Epoch 8/1000
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 

In [47]:
predictions = np.argmax(model.predict(x_testcnn),axis=1)

562/562 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [48]:
from sklearn.metrics import classification_report
report = classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.45      0.84      0.59      3167
           1       0.53      0.48      0.50      3065
           2       0.69      0.38      0.49      3002
           3       0.60      0.45      0.51      3114
           4       0.65      0.50      0.56      2568
           5       0.56      0.62      0.59      3066

    accuracy                           0.55     17982
   macro avg       0.58      0.54      0.54     17982
weighted avg       0.58      0.55      0.54     17982



## Modelo https://www.kaggle.com/code/shivamburnwal/speech-emotion-recognition?scriptVersionId=34958802&cellId=52

Acurácia de 0.79

In [10]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

In [13]:
model=Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(units=6, activation='softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model.summary()

c:\Users\alech\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 64, 256)        │        64,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 32, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32, 256)        │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 16, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 16, 128)        │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 8, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 8, 64)          │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 606,758 (2.31 MB)

 Trainable params: 606,182 (2.31 MB)

 Non-trainable params: 576 (2.25 KB)

In [14]:
from tensorflow.keras.utils import to_categorical

In [15]:
y_train_encoded = to_categorical(y_train, num_classes=6)
y_test_encoded = to_categorical(y_test, num_classes=6)

In [16]:
y_train_encoded.shape

(36508, 6)

In [17]:
callback = keras.callbacks.EarlyStopping(monitor='loss',patience=3)
history=model.fit(x_traincnn, y_train_encoded, batch_size=64, epochs=100, validation_data=(x_testcnn, y_test_encoded), callbacks=[callback])

Epoch 1/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 52s 84ms/step - accuracy: 0.2811 - loss: 1.7574 - val_accuracy: 0.5386 - val_loss: 1.2565
Epoch 2/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 40s 70ms/step - accuracy: 0.5225 - loss: 1.2441 - val_accuracy: 0.5898 - val_loss: 1.0677
Epoch 3/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 48s 83ms/step - accuracy: 0.5741 - loss: 1.1177 - val_accuracy: 0.5782 - val_loss: 1.0756
Epoch 4/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 44s 77ms/step - accuracy: 0.6073 - loss: 1.0399 - val_accuracy: 0.6053 - val_loss: 1.0588
Epoch 5/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 87s 86ms/step - accuracy: 0.6333 - loss: 0.9816 - val_accuracy: 0.6605 - val_loss: 0.9173
Epoch 6/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 82s 85ms/step - accuracy: 0.6644 - loss: 0.9091 - val_accuracy: 0.6931 - val_loss: 0.8246
Epoch 7/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 48s 85ms/step - accuracy: 0.6966 - loss: 0.8377 - val_accuracy: 0.7110 - val_loss: 0.7772
Epoch 8/100
571/571 ━━━━━━━━━━━━━━━━━━━━ 49s 85ms/step - accuracy: 0.7311 - loss: 0

In [18]:
predictions = np.argmax(model.predict(x_testcnn),axis=1)

562/562 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step


In [20]:
predictions_encoded = to_categorical(predictions, num_classes=6)

In [21]:
from sklearn.metrics import classification_report
report = classification_report(y_test_encoded, predictions_encoded)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.87      0.89      3167
           1       0.83      0.80      0.81      3065
           2       0.85      0.82      0.83      3002
           3       0.81      0.86      0.83      3114
           4       0.88      0.83      0.86      2568
           5       0.79      0.86      0.82      3066

   micro avg       0.84      0.84      0.84     17982
   macro avg       0.84      0.84      0.84     17982
weighted avg       0.84      0.84      0.84     17982
 samples avg       0.84      0.84      0.84     17982



              precision    recall  f1-score   support

           0       0.87      0.94      0.90       863
           1       0.75      0.58      0.65       290

   micro avg       0.85      0.85      0.85      1153
   macro avg       0.81      0.76      0.78      1153
weighted avg       0.84      0.85      0.84      1153
 samples avg       0.85      0.85      0.85      1153

In [22]:
MODEL_DIR_PATH = 'models'

In [23]:
if not os.path.isdir(MODEL_DIR_PATH):
    os.makedirs(MODEL_DIR_PATH)
model_path = os.path.join(MODEL_DIR_PATH, 'emotion_detection.keras')
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at models\emotion_detection.keras 
